In [2]:
import os

import numpy as np
import pandas as pd

In [3]:
participants = pd.read_csv("participant_info.csv")
participants

,sn,age,sex,category
0,1,44,M,Control
1,10,74,F,Patient
2,11,22,F,Control
3,12,28,F,Control
4,13,23,F,Control
5,14,22,F,Patient
6,15,57,F,Patient
7,16,39,F,Patient
8,17,39,M,Control
9,18,52,F,Patient


In [4]:
patients = participants[participants["category"] == "Patient"]
controls = participants[participants["category"] == "Control"]

In [5]:
alpha_electrodes = ["O1[9]", "O2[10]", "P3[7]", "P4[8]", "Fz[17]"]
beta_electrodes = ["F7[11]", "F8[12]", "T5[15]", "T6[16]", "Pz[18]"]
gamma_electrodes = ["Fz[17]", "Pz[18]", "O1[9]", "O2[10]", "Cz[19]"]

In [6]:
def calculate_power(data, freq_range):
    lower, upper = freq_range
    fft_result = np.fft.fft(data)
    frequencies = np.fft.fftfreq(len(data), 1 / 100)

    # Filter frequencies within the range specified
    mask = (frequencies >= lower) & (frequencies <= upper)
    filtered_fft_result = fft_result[mask]
    # Calculate power spectrum
    power_spectrum = (np.abs(filtered_fft_result) ** 2) / len(filtered_fft_result) ** 2

    return power_spectrum

In [7]:
dataset = {"id": []}

for t in ["Alpha", "Beta", "Gamma"]:
    for i in range(51):
        dataset[f"{t}_{i}"] = []

dataset["outcome"] = []

eeg_data_dir = "Output EEG Data"
for prtcpnt in os.listdir(eeg_data_dir):
    outcome = "P" if int(prtcpnt) in list(patients["sn"]) else "C"
    dataset["id"].append(f"{prtcpnt}")
    dataset["outcome"].append(outcome)
    
    for phase in range(1, 3):
        eeg_data = pd.read_csv(f"{eeg_data_dir}/{prtcpnt}/Phase {phase}.csv")

        if phase == 1:
            alpha_electrode_data = list(eeg_data[alpha_electrodes[0]])
            alpha_power_data = calculate_power(alpha_electrode_data, (8, 13))

            for i in range(51):
                dataset[f"Alpha_{i}"].append(alpha_power_data[i])

        else:
            beta_electrode_data = list(eeg_data[beta_electrodes[-1]])
            beta_power_data = calculate_power(beta_electrode_data, (12, 30))
            gamma_electrode_data = list(eeg_data[gamma_electrodes[1]])
            gamma_power_data = calculate_power(gamma_electrode_data, (30, 100))

            for i in range(51):
                dataset[f"Beta_{i}"].append(beta_power_data[i])
                dataset[f"Gamma_{i}"].append(gamma_power_data[i])

In [8]:
len(dataset["id"]), len(dataset["outcome"]), len(dataset["Alpha_0"]), len(dataset["Beta_0"])

(27, 27, 27, 27)

In [9]:
dataset_df = pd.DataFrame(dataset)
dataset_df.to_csv("Frequency Analysis Dataset/Dataset.csv")

In [12]:
# Import necessary libraries
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, average_precision_score

# Load your own dataset (replace 'your_dataset.csv' with your file path)
dataset_path = 'Frequency Analysis Dataset/Dataset.csv'
df = pd.read_csv(dataset_path)

# Assume the last column is the target variable
X = df.iloc[:, :-1]
y = df.iloc[:, -1]

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create a Random Forest Classifier
rf_classifier = RandomForestClassifier(n_estimators=100, random_state=42)

# Train the classifier on the training data
rf_classifier.fit(X_train, y_train)

# Make predictions on the test set
y_pred = rf_classifier.predict(X_test)

# Calculate evaluation metrics
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, labels=["P", "C"], pos_label="P")
recall = recall_score(y_test, y_pred, labels=["P", "C"], pos_label="P")
f1 = f1_score(y_test, y_pred, labels=["P", "C"], pos_label="P")
roc_auc = roc_auc_score(y_test, rf_classifier.predict_proba(X_test)[:, 1])
pr_auc = average_precision_score(y_test, rf_classifier.predict_proba(X_test)[:, 1],  pos_label="P")

# # Display the results
print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1:.4f}")
print(f"AUC-ROC: {roc_auc:.4f}")
print(f"AUC-PR: {pr_auc:.4f}")


Accuracy: 0.6667
Precision: 0.6667
Recall: 1.0000
F1 Score: 0.8000
AUC-ROC: 0.3750
AUC-PR: 0.6667
